In [20]:
# Cell 1: Imports
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import math
import os
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path


# Create directory for datasets
os.makedirs('uploaded_dataset', exist_ok=True)

In [21]:
# Cell 2: Upload widget
# uploader = widgets.FileUpload(
#     accept='.mp4,.avi,.mov', 
#     multiple=False
# )
# display(uploader)

# import ipywidgets as widgets
# from IPython.display import display

# label_selector = widgets.Dropdown(
#     options=[('Normal', 0), ('Abnormal', 1)],
#     description='Gait Type:',
#     style={'description_width': 'initial'}
# )

# display(label_selector)


In [22]:
# Cell 3: Save uploaded video
video_path = None

if uploader.value:
    uploaded_file = uploader.value[0]
    video_name = uploaded_file.name
    video_path = f"uploaded_dataset/{video_name}"
    with open(video_path, 'wb') as f:
        f.write(uploaded_file.content)
    print(f"Saved to: {video_path}")

Saved to: uploaded_dataset/Parkinsonian Gait (Dr. Yehia Mishriki).mp4


In [17]:
# Cell 4: Create Dataset
def extract_gait_features(video_path, label=None, output_csv=True):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False)
    scale_factor = 0.3  # Lower scale for faster processing
    frame_skip = 5      # Process every 5th frame
    max_duration = 10   # Stop after 10 seconds of video

    def resize_image(image, scale_factor):
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        return cv2.resize(image, (new_width, new_height))

    def calculate_angle(a, b, c):
        a, b, c = np.array(a), np.array(b), np.array(c)
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians * 180.0 / np.pi)
        return 360 - angle if angle > 180.0 else angle

    def get_coords(landmarks, index):
        return (landmarks[index].x, landmarks[index].y)

    RIGHT_HIP, RIGHT_KNEE, RIGHT_ANKLE = 24, 26, 28
    LEFT_HIP, LEFT_KNEE, LEFT_ANKLE = 23, 25, 27

    cap = cv2.VideoCapture(video_path)
    prev_angle, prev_angular_velocity, prev_time = None, None, 0
    data_list = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        if frame_idx % frame_skip != 0:
            continue

        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        if current_time > max_duration:
            break

        frame = resize_image(frame, scale_factor)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            rh, rk, ra = get_coords(lm, RIGHT_HIP), get_coords(lm, RIGHT_KNEE), get_coords(lm, RIGHT_ANKLE)
            lh, lk, la = get_coords(lm, LEFT_HIP), get_coords(lm, LEFT_KNEE), get_coords(lm, LEFT_ANKLE)

            rk_angle = calculate_angle(rh, rk, ra)
            lk_angle = calculate_angle(lh, lk, la)

            v1 = (rk[0] - ra[0], rk[1] - ra[1])
            v2 = (rh[0] - ra[0], rh[1] - ra[1])
            dot = v1[0]*v2[0] + v1[1]*v2[1]
            mag1 = math.sqrt(v1[0]**2 + v1[1]**2)
            mag2 = math.sqrt(v2[0]**2 + v2[1]**2)
            angle_rad = math.acos(dot / (mag1 * mag2 + 1e-6))
            angle_deg = math.degrees(angle_rad)

            time_diff = current_time - prev_time
            angular_velocity = None
            if prev_angle is not None and time_diff > 0:
                angular_velocity = (angle_deg - prev_angle) / time_diff

            linear_acceleration = None
            if prev_angular_velocity is not None and angular_velocity is not None:
                linear_acceleration = (angular_velocity - prev_angular_velocity)

            prev_angle = angle_deg
            prev_angular_velocity = angular_velocity
            prev_time = current_time

            data_list.append({
                'time': current_time,
                'right_knee_angle': rk_angle,
                'left_knee_angle': lk_angle,
                'angular_velocity': angular_velocity,
                'linear_acceleration': linear_acceleration,
                'label': label
            })

    cap.release()
    pose.close()

    df = pd.DataFrame(data_list)
    csv_name = Path(video_path).stem + '_gait.csv'
    if output_csv:
        df.to_csv(f'uploaded_dataset/{csv_name}', index=False)
        print(f"Saved: uploaded_dataset/{csv_name}")
    return df


In [ ]:
# Cell 5: Run feature extraction
if video_path:
    df = extract_gait_features(video_path, label=None)
    display(df.head())

C:\Users\Sandra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved: uploaded_dataset/Parkinsonian Gait (Dr. Yehia Mishriki)_gait.csv


,time,right_knee_angle,left_knee_angle,angular_velocity,linear_acceleration,label
0,0.133333,171.179357,176.989553,NaN,NaN,0
1,0.300000,173.274225,166.805141,-5.878254,NaN,0
2,0.466667,169.958672,172.133824,9.501371,15.379625,0
3,0.633333,170.085910,167.024400,-0.358759,-9.860129,0
4,0.800000,170.255812,166.111920,-0.488355,-0.129597,0


In [19]:
# Cell 6: Prediction
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load all labeled gait datasets
all_files = list(Path("uploaded_dataset").glob("*_gait.csv"))
full_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)

# Drop rows with NaN
full_df.dropna(inplace=True)

# Features and labels
X = full_df[['right_knee_angle', 'left_knee_angle', 'angular_velocity', 'linear_acceleration']]
y = full_df['label']

# Split into train/test (or train all)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict current uploaded video
input_features = df[['right_knee_angle', 'left_knee_angle', 'angular_velocity', 'linear_acceleration']].mean().values.reshape(1, -1)
prediction = model.predict(input_features)

# Display result
print("Prediction:", "Normal" if prediction[0] == 0 else "Abnormal")



Prediction: Normal


C:\Users\Sandra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
